In [2]:
clear all;

global ipynb = 'ttt';

source('clearest-nn.m');
source('utils-logging.m');
source('utils-training.m');
source('game-ttt.m');

% log2file(tmp('log'));

##########################################

[winner,s] = play1(@randompolicy, @randompolicy)
oh18 = game2oh18(s)'
oh27 = game2oh27(s)'
play(100, @randompolicy);


ans = THE CLEAREST NEURAL NETWORK FRAMEWORK BY UNDWAD
ans = TIC-TAC-TOE GAME
winner =  1
s =

   2   0   1
   1   2   1
   2   0   1

oh18 =

   0   1   1   0   0   1   0   0   0   1   0   0   1   0   1   0   1   0

oh27 =

 Columns 1 through 26:

  0  0  0  1  0  1  0  0  0  0  1  0  0  0  0  1  1  1  1  0  1  0  1  0  0  0

 Column 27:

  0

playing 100 times randompolicy vs randompolicy
wins = [58 23]
draws = 19


In [3]:
### MODEL ###

global states turns V pi Q;

function enum(s = game(), t = tic())
    global states turns V pi;
    idx = state2num(s);
    showlog(1, 50, '%d %s %s', idx, mat2str(s))
    if turns(idx) == 0
        states(:,:,idx) = s;
        turns(idx)      = player(s);
        aaa             = actions(s);
        na              = count(aaa);
        if na > 0
            pi(aaa,idx) = softmax(rand(na,1));
        end
        for a = aaa
            enum(game(s, a), t);
        end
    end
end

rand('state', 1);

path   = tmp('dataset.mat')
exists = logical(exist(path))
if exists
    load('-binary', path, 'states', 'turns', 'V', 'pi', 'Q');
else
    n      = 3^9;
    states = zeros(3,3,n);
    turns  = zeros(1,n);
    V      = zeros(1,n);
    pi     = zeros(9,n);
    Q      = zeros(9,n);
    printstart();
    enum();
    Q = pi;
    save('-binary', path, 'states', 'turns', 'V', 'pi', 'Q');
    printend(path);    
end

num_s = count(states);
printvar('num_s');
printvar('player(states(:,:,100))');
printvar('turns(100)');
printvar('states(:,:,100)');
printvar('V(100)');
printvar('pi(:,100)');
printvar('sum(pi(:,100))');
printvar('player(states(:,:,101))');
printvar('turns(101)');
printvar('states(:,:,101)');
printvar('V(101)');
printvar('pi(:,101)');
printvar('sum(pi(:,101))');

num_game_s = 0;
for i=1:num_s
    if turns(i) != 0
        s = states(:,:,i);
        p = player(s);
        assert(p == turns(p));
        s_ = num2state(i);
        assert(all(all(s_ == s)));
        num_game_s += 1;
    end
end

printvar('num_game_s');


path = tmp/tic-tac-toe.dataset.mat
exists = 1
num_s = 19683
player(states(:,:,100)) = 1
turns(100) = 1
states(:,:,100) = [0 0 0;0 1 0;2 0 0]
V(100) = 0
pi(:,100) = [0.109167230004231;0.160779337725954;0;0.136495405475451;0;0.138484815816882;0.148083386090299;0.189783691910609;0.117206131078737]
sum(pi(:,100)) = 1.000000
player(states(:,:,101)) = 2
turns(101) = 2
states(:,:,101) = [1 0 0;0 1 0;2 0 0]
V(101) = 0
pi(:,101) = [0;0.154605399819868;0;0.122024599336157;0;0.272938546704486;0.134159348916976;0.139850816077742;0.176421286415385]
sum(pi(:,101)) = 1.000000
num_game_s = 6046


In [4]:
### GAME DYNAMICS - p(s',r|s,a) ###

function a = pipolicy(s)
    global pi;
    idx   = state2num(s);
    [~,a] = max(pi(:,idx));
end

# next        = [ [next_idx; reward], ... ]
# probability = 1/count(next)
function next = dynamics(s, a) 
    # 3 = 2 + 1 = smallest terminal state index 
    global states;
    s = game(s,a);
    if iswin(s,a)  
        next = [ 3; 1 ]; # learning player wins
    elseif isover(s) 
        next = [ 3; 0 ]; # draw
    else
        aaa  = actions(s);
        na   = count(aaa);
        next = zeros(2,na);
        for i = 1:na
            a   = aaa(i);
            s_  = game(s,a);
            idx = state2num(s_);
            if iswin(s_,a)   
                next(:,i) = [ 3;  -1 ]; # other player wins
            elseif isover(s_)
                next(:,i) = [ 3;   0 ]; # draw
            else
                next(:,i) = [ idx; 0 ]; # continue playing
            end  
        end
    end
end

do 
    s = pick(states);
until nnz(s) > 0;
s
p    = player(s)
aaa  = actions(s)
a    = pick(aaa)
next = dynamics(s, a)

play(100, @randompolicy, @pipolicy);
play(100, @pipolicy, @randompolicy);


s =

   1   1   0
   2   2   1
   2   1   0

p =  2
aaa =

   7   9

a =  9
next =

   3
  -1

playing 100 times randompolicy vs pipolicy
wins = [66 25]
draws = 9
playing 100 times pipolicy vs randompolicy
wins = [65 25]
draws = 10


<img align="left" src="assets/dp-policy-iter.png">

In [5]:
### DYNAMIC PROGRAMMING POLICY ITERATION ###

global theta gamma;

function vvv = bellman_values(s, a)
    global V gamma;
    next   = dynamics(s, a);
    n      = count(next);
    vvv    = zeros(1,n);
    for i  = 1:n 
        [next_idx, reward] = split(next(:, i));
        vvv(i)             = 1/n * (reward + gamma*V(next_idx));
    end
end

function bellman_update(idx)
    global states V pi;
    s     = states(:,:,idx);
    ppp   = pi(:,idx);
    v     = 0;
    for a = actions(s) 
        vvv = ppp(a) .* bellman_values(s, a);
        v  += sum(vvv);  
    end
    V(idx) = v;
end

function greedify_policy(idx)
    global states V pi;
    s      = states(:,:,idx);
    max_qa = 0;
    best_a = 0;
    for a = actions(s) 
        vvv = bellman_values(s, a);
        qa  = sum(vvv); 
        if qa > max_qa
            max_qa = qa;
            best_a = a;
        end
    end
    if best_a > 0
        pi(:,idx)      = 0;
        pi(best_a,idx) = 1;
    end
end

function evaluate_policy(p)
    global states turns V theta;
    do # global loop until delta < theta
        delta = 0;
        num_s = count(states);
        for idx = 1:num_s # iter all states
            if turns(idx) == p # learning player's state
                v = V(idx);
                bellman_update(idx);
                delta = max(delta, abs(v - V(idx)));
                showlog(1, 50, '%s %d %d %s %f', 'evaluate_policy', p, idx, dec2base(idx-1,3), delta);
            end
        end
    until delta < theta;
end

function stable = improve_policy(p)
    global states turns pi;
    stable = true;
    num_s = count(states);
    for idx = 1:num_s # iter all states
        if turns(idx) == p # learning player's state
            ppp = pi(:,idx);
            greedify_policy(idx);
            if any(ppp != pi(:,idx))
                stable = false;
            end
            showlog(1, 50, '%s %d %d %s %s', 'improve_policy', p, idx, dec2base(idx-1,3), bool2yesno(stable));
        end
    end
end

function policy_iteration(p)
    do
        evaluate_policy(p);
        stable = improve_policy(p);
    until stable;
end

theta = 0.0001;
gamma = 0.5;

path   = tmp('dynamic-prog-policy-iter.mat')
exists = logical(exist(path))
if exists
    load('-binary', path, 'V', 'pi');
else
    printstart();
    policy_iteration(1);
    policy_iteration(2);
    save('-binary', path, 'V', 'pi');
    printend(path);    
end

play(100, @pipolicy, @randompolicy);
play(100, @randompolicy, @pipolicy);
play(100, @pipolicy);


path = tmp/tic-tac-toe.dynamic-prog-policy-iter.mat
exists = 1
playing 100 times pipolicy vs randompolicy
wins = [100 0]
draws = 0
playing 100 times randompolicy vs pipolicy
wins = [1 94]
draws = 5
playing 100 times pipolicy vs pipolicy
wins = [0 0]
draws = 100


<img align="left" src="assets/dp-value-iter.png">

In [6]:
### DYNAMIC PROGRAMMING VALUE ITERATION ###

function bellman_update(idx)
    global states V pi;
    s     = states(:,:,idx);
    ppp   = pi(:,idx);
    max_v = 0;
    for a = actions(s) 
        vvv   = ppp(a) .* bellman_values(s, a);
        max_v = max(sum(vvv), max_v);
    end
    V(idx) = max_v;
end

theta = 0.0001;
gamma = 0.5;

path   = tmp('dynamic-prog-value-iter.mat')
exists = logical(exist(path))
if exists
    load('-binary', path, 'V', 'pi');
else
    printstart();
    policy_iteration(1);
    policy_iteration(2);
    save('-binary', path, 'V', 'pi');
    printend(path);    
end

play(100, @pipolicy, @randompolicy);
play(100, @randompolicy, @pipolicy);
play(100, @pipolicy);


path = tmp/tic-tac-toe.dynamic-prog-value-iter.mat
exists = 1
playing 100 times pipolicy vs randompolicy
wins = [100 0]
draws = 0
playing 100 times randompolicy vs pipolicy
wins = [2 96]
draws = 2
playing 100 times pipolicy vs pipolicy
wins = [0 100]
draws = 0


<img align="left" src="assets/sarsa-iter.png">

In [8]:
### SARSA (ON-POLICY TEMPORAL DIFFERENCE CONTROL) ###

global epsilon alpha gamma Q;

function [a,q] = qpolicy(s)
    global Q;
    aaa   = actions(s);
    idx   = state2num(s);
    qqq   = Q(:,idx)(aaa);
    [q,i] = max(qqq);
    a     = aaa(i);
end

function a = epsilon_greedy_qpolicy(s)
    global epsilon;
    if rand() < epsilon
        a = pick(actions(s));
    else
        a = qpolicy(s);
    end
end

function winner = sarsa_episode(target, p)
    global alpha gamma Q;
    winner = [];
    if p == 1
        s = game();
        a = epsilon_greedy_qpolicy(s);
    elseif p == 2
        s = game();
        a = randompolicy(s);
        s = game(s, a);
        a = epsilon_greedy_qpolicy(s);        
    end
    do
        idx    = state2num(s);
        q      = Q(a, idx);
        next_s = game(s, a);               # learning player moves
        if iswin(next_s, a)                # learning player wins
            reward = 1;
            next_q = 0;
            winner = p;
        elseif isover(next_s)              # draw
            reward = 0;
            next_q = 0;
            winner = 0;
        else                               # continue
            oppo_a = randompolicy(next_s); 
            next_s = game(next_s, oppo_a); # other player moves
            if iswin(next_s, oppo_a)       # other player wins
                reward = -1;
                next_q = 0;
                winner = player(next_s, oppo_a);
            elseif isover(next_s)          # draw
                reward = 0;
                next_q = 0;
                winner = 0;
            else                           # continue
                reward = 0;
                [next_a,next_q] = target(next_s);
            end  
        end  
        Q(a, idx) = q + alpha*(reward + gamma*next_q - q);
        s = next_s;
        a = next_a;
    until !isempty(winner);    
end

function [a,q] = sarsa_epsilon_greedy_target(s)
    global Q;
    a = epsilon_greedy_qpolicy(s);
    q = Q(a, state2num(s));
end

function sarsa_iteration(target, p, n)
    wins = zeros(3,1);
    for i = 1:n
        showlog(1, 80, 'sarsa_iteration for %d, %d of %d, %s', p, i, n, mat2str(wins));
        winner = sarsa_episode(target, p);
        wins(winner+1) += 1;
    end
end

theta   = 0.0001;
epsilon = 0.1;
alpha   = 0.1;
gamma   = 0.5;
iters   = 10000;

path   = tmp('sarsa.mat')
exists = logical(exist(path))
if exists
    load('-binary', path, 'Q');
else
    printstart();
    sarsa_iteration(@sarsa_epsilon_greedy_target, 1, iters);
    sarsa_iteration(@sarsa_epsilon_greedy_target, 2, iters);
    save('-binary', path, 'Q');
    printend(path);    
end

play(100, @qpolicy, @randompolicy);
play(100, @randompolicy, @qpolicy);
play(100, @qpolicy);


path = tmp/tic-tac-toe.sarsa.mat
exists = 1
playing 100 times qpolicy vs randompolicy
wins = [97 1]
draws = 2
playing 100 times randompolicy vs qpolicy
wins = [3 89]
draws = 8
playing 100 times qpolicy vs qpolicy
wins = [0 0]
draws = 100


<img align="left" src="assets/q-learning-iter.png">

In [9]:
### Q-LEARNING \ SARSAMAX (OFF-POLICY TEMPORAL DIFFERENCE CONTROL) ###

function [a,q] = sarsa_max_target(s)
    global Q;
    aaa   = actions(s);
    qqq   = Q(:, state2num(s));
    q     = max(qqq(aaa));
    a     = epsilon_greedy_qpolicy(s);
end

theta   = 0.0001;
epsilon = 0.1;
alpha   = 0.1;
gamma   = 0.5;
iters   = 10000;

path   = tmp('sarsa-max.mat')
exists = logical(exist(path))
if exists
    load('-binary', path, 'Q');
else
    printstart();
    sarsa_iteration(@sarsa_max_target, 1, iters);
    sarsa_iteration(@sarsa_max_target, 2, iters);
    save('-binary', path, 'Q');
    printend(path);    
end

play(100, @qpolicy, @randompolicy);
play(100, @randompolicy, @qpolicy);
play(100, @qpolicy);


path = tmp/tic-tac-toe.sarsa-max.mat
exists = 1
playing 100 times qpolicy vs randompolicy
wins = [100 0]
draws = 0
playing 100 times randompolicy vs qpolicy
wins = [2 90]
draws = 8
playing 100 times qpolicy vs qpolicy
wins = [0 0]
draws = 100


In [10]:
### EXPECTED SARSA \ SARSAMEAN (OFF-POLICY TEMPORAL DIFFERENCE CONTROL) ###

function [a,q] = sarsa_mean_target(s)
    global Q;
    aaa   = actions(s);
    qqq   = Q(:, state2num(s));
    q     = mean(qqq(aaa));
    a     = epsilon_greedy_qpolicy(s);
end

theta   = 0.0001;
epsilon = 0.1;
alpha   = 0.1;
gamma   = 0.5;
iters   = 10000;

path   = tmp('sarsa-mean.mat')
exists = logical(exist(path))
if exists
    load('-binary', path, 'Q');
else
    printstart();
    sarsa_iteration(@sarsa_mean_target, 1, iters);
    sarsa_iteration(@sarsa_mean_target, 2, iters);
    save('-binary', path, 'Q');
    printend(path);    
end

play(100, @qpolicy, @randompolicy);
play(100, @randompolicy, @qpolicy);
play(100, @qpolicy);


path = tmp/tic-tac-toe.sarsa-mean.mat
exists = 1
playing 100 times qpolicy vs randompolicy
wins = [94 3]
draws = 3
playing 100 times randompolicy vs qpolicy
wins = [1 87]
draws = 12
playing 100 times qpolicy vs qpolicy
wins = [0 0]
draws = 100


<img align="left" src="assets/td-advantage-actor-critic.png">

In [11]:
### TEMPORAL DIFFERENCE ADVANTAGE ACTOR-CRITIC ###

% rand('state', 1);

global actor critic gamma;

function delta = criticize(prev_s, next_s, reward, winner)
    global critic gamma;
    if isempty(winner)
        x            = game2onehot(next_s);
        [~,y]        = forward(critic, x);
    else
        y            = 0;
    end
    y                = reward + gamma*y;
    x                = game2onehot(prev_s);
    [critic,z]       = forward(critic, x);
    dE               = gradient(critic, z, y);
    [critic, ggg, ~] = backward(critic, z, dE);
    [critic, ggg]    = optimize_gradient(critic, ggg, 1);
    critic           = update(critic, ggg);
    delta            = y - z; 
end

function [a,z] = actorpolicy(s)
    global actor;
    x         = game2onehot(s);
    [actor,z] = forward(actor, x);
    aaa       = actions(s); 
    z_        = zeros(9,1);
    x         = actor.XXX{end}(aaa);
    z_(aaa)   = softmax(x);
    f         = mnrnd(1,z_);
    a         = find(f); 
end

function learn2act(a, z, delta, eps=1e-8)
    global actor;
    y               = zeros(size(z));
    y(a)            = delta;
    dE              = gradient(actor, z, y);
    [actor, ggg, ~] = backward(actor, z, dE);
    [actor, ggg]    = optimize_gradient(actor, ggg, 1);
    actor           = update(actor, -ggg);
end

function winner = episode(p)
    global actor critic gamma;
    winner = [];
    s      = game();
    if p == 2
        a = randompolicy(s);
        s = game(s, a); 
    end
    do
        prev_s = s;
        [a,z]  = actorpolicy(s); 
        s      = game(s, a);        # learning player moves
        if iswin(s, a)                 # learning player wins
            reward = 1;
            winner = p;
        elseif isover(s)               # draw
            reward = 0;
            winner = 0;
        elseif player(s) == p          # invalid move
            error('invalid move');
        else                           # continue
            a = randompolicy(s); 
            s = game(s, a);            # other player moves
            if iswin(s, a)             # other player wins
                reward = -1;
                winner = player(s, a);
            elseif isover(s)           # draw
                reward = 0;
                winner = 0;
            else                       # continue
                reward = 0;
            end  
        end  
        delta = criticize(prev_s, s, reward, winner);
        learn2act(a, z, delta);
    until !isempty(winner);    
end

function ratio = iteration(p, n, eps=1e-8)
    global critic actor aEEE cEEE;
    RATIO = zeros(1,n);
    wins  = zeros(3,1);
    for i = 1:n
        winner          = episode(p);
        wins(winner+1) += 1;
        ratio           = wins(1+1)/wins(2+1);
        RATIO(i)        = ratio;
        critic_gradnorm = getunit(critic.optimizers,'gradient_clipping').norm;
        actor_gradnorm  = getunit( actor.optimizers,'gradient_clipping').norm;
        critic_updratio = getunit(critic.optimizers,'stats').ratio;
        actor_updratio  = getunit( actor.optimizers,'stats').ratio;
        showlog(1, 80, 'player %d episode %d of %d, gradnorm %f vs %f, updratio %f vs %f, wins %s, %f', 
                        p, i, n, critic_gradnorm, actor_gradnorm, critic_updratio, actor_updratio, mat2str(wins), ratio);
    end
    figure('Position', [0 0 1000 400]);
    plot(1:n, RATIO, 'b');
    title('win ratio history');
end

gamma  = 1;
critic = model(18, {'dense', 1});
critic = optimization(critic, {'adam', 0.1},   {'gradient_clipping', 0.9}, 'stats');
critic = objective(critic, 'mse');
actor  = model(18, {'dense', 9}, 'softmax');
actor  = optimization(actor,  {'adam', 0.03},  {'gradient_clipping', 0.9}, 'stats');
actor  = objective(actor, 'logloss');

% printmodel('critic');
% printmodel('actor');

[winner,s] = play1(@randompolicy, @actorpolicy)
play(100, @randompolicy, @actorpolicy);

path   = tmp('actor-critic.mat')
exists = logical(exist(path))
if exists
    load('-binary', path, 'actor', 'critic');
else
    printstart();
    ratio = iteration(2, 1000);
    save('-binary', path, 'actor', 'critic');
    printend(sprintf('%s %f', path, ratio));    
end

play(100, @randompolicy, @actorpolicy);


winner =  1
s =

   1   1   1
   2   1   2
   2   1   2

playing 100 times randompolicy vs actorpolicy
wins = [52 38]
draws = 10
path = tmp/tic-tac-toe.actor-critic.mat
exists = 1
playing 100 times randompolicy vs actorpolicy
wins = [5 85]
draws = 10


In [13]:
play(1000, @randompolicy, @randompolicy);
play(1000, @randompolicy, @pipolicy);
play(1000, @randompolicy, @qpolicy);
play(1000, @randompolicy, @actorpolicy);
play(1000, @pipolicy,     @actorpolicy);
play(1000, @qpolicy,      @actorpolicy);
play(1000, @pipolicy,     @qpolicy);
play(1000, @qpolicy,      @pipolicy);

playing 1000 times randompolicy vs randompolicy
wins = [579 291]
draws = 130
playing 1000 times randompolicy vs pipolicy
wins = [41 927]
draws = 32
playing 1000 times randompolicy vs qpolicy
wins = [28 852]
draws = 120
playing 1000 times randompolicy vs actorpolicy
wins = [124 782]
draws = 94
playing 1000 times pipolicy vs actorpolicy
wins = [234 764]
draws = 2
playing 1000 times qpolicy vs actorpolicy
wins = [356 0]
draws = 644
playing 1000 times pipolicy vs qpolicy
wins = [1000 0]
draws = 0
playing 1000 times qpolicy vs pipolicy
wins = [0 0]
draws = 1000
